In [ ]:
#Importing Libraries all of them are pre install on notebooks, if you use other environemnt you can install like next cell
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import json
import os


In [ ]:
import sys
!{sys.executable} -m pip install azure-cognitiveservices-vision-customvision

In [ ]:
print("Starting process")

In [ ]:
#Variables section

#bing variables
search_url = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"
search_term = "person"
tagname = "person" #Bing search term how to find pictures can be synomyns
bing_subscription_key = "xxx"

#Customvision variables
training_key = "2xxx"
ENDPOINT="https://southcentralus.api.cognitive.microsoft.com"
project= "ee9fd341-32ad-499c-a0c7-80924f8a51ba"



In [ ]:

import warnings
import contextlib
import requests
from urllib3.exceptions import InsecureRequestWarning


old_merge_environment_settings = requests.Session.merge_environment_settings

@contextlib.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

In [ ]:
headers = {"Ocp-Apim-Subscription-Key" : bing_subscription_key}
params  = {"q": search_term, "license": "All", "imageType": "photo","count" : "200","safeSearch": "Off"}
#You can change all this parameters according to api documentation to get next 200 or enable the SafeSearch
# https://docs.microsoft.com/en-us/rest/api/cognitiveservices/bing-web-api-v7-reference

In [ ]:
with no_ssl_verification():
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

In [ ]:
import os
import glob
# define the name of the directory to be created
path = "./" + tagname
if os.path.exists(path):
    print("folder exists removing content")
    for root, dirs, files in os.walk(path):
        for f in files:
            os.remove(path + '/' + f)
try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)

In [ ]:
import urllib.request
opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)
count=0
for contenturl in search_results['value']:
    print (contenturl['contentUrl'])
    count= count+1
    try:
        f = open( tagname +'/'+ str(count) + '.jpg','wb')
        f.write(urllib.request.urlopen(contenturl['contentUrl']).read())
        f.close()
    except:
        pass
    

In [ ]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry


trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)
#trainer = training_api.TrainingApi(training_key)

# Create a new project
def resize(pathimg):
  basewidth = 1280
  img = Image.open(pathimg)
  wpercent = (basewidth/float(img.size[0]))
  hsize = int((float(img.size[1])*float(wpercent)))
  img2 = img.resize((basewidth,hsize), Image.ANTIALIAS)
  rgb_im = img2.convert('RGB')
  rgb_im.save(pathimg) 
  #img.save(pathimg) 

#print ("Creating project...")
def checkTag(tagname):
    trainer.get_project(project)
    for i in trainer.get_tags(project):
        if tagname == i.name:
            print (i.id)
            return i.id
    return '0'

def createTag(tagname):
    trainer.get_project(project)
    return trainer.create_tag(project,tagname).id

def CreateOrGet(tagname):
    if checkTag(tagname) == '0':
        return createTag(tagname)
    else:
        return checkTag(tagname)

def uploadimage(picture,tagname):
    #with open(os.fsdecode(picture), mode="rb") as img_data: 
        #v= trainer.create_images_from_data(project, img_data)
    my_tag = CreateOrGet(tagname)
    print(my_tag)
    images_to_upload = []
    with open(picture, mode="rb") as image_contents:
       images_to_upload.append(ImageFileCreateEntry(name=picture, contents=image_contents.read()))
    v = trainer.create_images_from_files(project,images=images_to_upload,tag_ids=[my_tag] )
    return v

def uploadimagememory(picture,tagname):
    my_tag = CreateOrGet(tagname)
    v = trainer.create_images_from_data(project,picture,tag_ids=[my_tag])    
    return v

def runupload(picture,tag,memory=False):
    if memory == False:
        print (picture)
        t = str(uploadimage(picture,tag).is_batch_successful)
    else:
        t = str(uploadimagememory(picture,tag).is_batch_successful)
    return (t)

In [ ]:
import os
print(tagname)
list1 = os.listdir("./" + tagname)
print(list1)


In [ ]:
for myfile in list1:
    mypath ='./' + tagname + '/'+ myfile
    print(mypath)
    try:
        resize(mypath)
        with no_ssl_verification():
            print(runupload(mypath,tagname))
    except:
        print("failed" + path)
        pass